# Ett test att göra en Cu@Ag particel och lyckas "se" den med HyperSpy

Ni kommer behöva antingen göra två egna spectrum eller ladda ned LucPureAg.msa & LucPureCu.msa. 

In [1]:
%matplotlib qt
import hyperspy.api as hs
import math
import numpy as np
import matplotlib.pyplot as plt

C:\Users\Lucas\anaconda3\lib\site-packages\pyUSID\viz\__init__.py:16: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  warn('Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. '


Jonas Metod

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb  1 16:12:36 2021
@author: Jonas
Method to create matrix with sphere
"""
import math;
import numpy as np;
class EdxMat:
    def __init__(self,size,r: float,dens:float,l: float):
        '''size: defines the 'picture' size as size x size pixels\n
        r: defines the radius of the spherical particle\n
        dens: density of the material the particle is made of\n
        l: pixel size in the same unit as r (i.e. the area is l^2)'''
        self.size=size;
        self.r=r;
        self.dens=dens;
        self.l=l;
        rred=r/l;
        self.mid= float((size-1)/2);#-1 because indices start from zero
        #The constructor now constructs a matrix using the input arguments
        mat=np.zeros((size,size));
        self.mat=mat;
        for i in range(size):
            for j in range(size):
                if rred**2>=((i-self.mid)**2+(j-self.mid)**2):
                    mat[i][j]=2*dens*self.thick(i,j);
        
    def thick(self,n,m):
        nr=n-self.mid;
        mr=m-self.mid;
        l=self.l;
        r=self.r;
        return math.sqrt(r**2-(nr*l)**2-(mr*l)**2);

In [3]:
x1=EdxMat(50,20.0,10.49,1.0)  #Den första hela "partikeln" som sen kommer bli skal (Ag)
x2=EdxMat(50,15.0,10.49,1.0)  #Den andra mindre "partikeln" som kommer ta bort insidan så det blir ett skal (Ag)
x3=EdxMat(50,15,8.96,1)       #Den faktiska mindre "partikeln" som är koppar kärnan (Cu)
shell = x1.mat-x2.mat         #Skapar skalet (Ag)
core = x3.mat                 #Skapar kärnan (Cu)

Färdigställande av partikel:

In [4]:
'''
Det var tydligen rätt viktigt att spectrumen var gjorda på samma sätt för att man skulle kunna se skal och kärna i 
samma bild. Annars blev det att kärnan eller skalet fanns där men inte syntes varken i bilden eller spektrat då
intensiteten var för låg. 
'''
sAg = hs.load("LucPureAg.msa",signal_type="EDS_TEM")
sCu = hs.load("LucPureCu.msa",signal_type="EDS_TEM") 

#Tomma matriser med rätt dimensioner
arr = np.empty((50,50,2048)) 
core3d = arr.copy() # .copy() för att annars pekar de på varandra och core och shell blandas
shell3d = arr.copy()
Ag = sAg.data #Extraherar bara spectrumen från .msa-filerna
Cu = sCu.data


for i in range(0,50):
    for j in range(0,50):
        #Här fylls de tomma core och shell matriserna med spectrum*intensitet från sfärmatriserna
        core3d[i,j,0:2048]=core[i,j]*Cu
        shell3d[i,j,0:2048]=shell[i,j]*Ag

In [5]:
#Nu slås de core och shell ihop och blir den fullständiga particeln:
matr = shell3d + core3d
maxval = np.max(matr)
#matr = (matr/maxval*255).astype(np.uint8)  #Denna behövs nog inte till hyperspy men behövs om man ska göra en egentlig bild tror jag.

292535.35803134943


Baserat på Filips spektrum-metod:

In [6]:
p = hs.signals.Signal1D(matr) #Läser in matrisen som en hyperspysignal

In [7]:
#Ett gäng metadata som kanske inte är nödvändigt men underlättar
p.set_signal_type("EDS_TEM") 
p.axes_manager.signal_axes[0].units = 'keV' #OBS! Enheten på energiaxeln är viktig för att kunna plotta
p.axes_manager[0].name = 'y'
p.axes_manager[1].name = 'x'
p.axes_manager['x'].units = 'nm'
p.axes_manager['y'].units = 'nm'
p.axes_manager[-1].name = 'E'
p.add_elements(['Ag','Cu']) #Lägger in element igen tydligen förs de inte med 
p.plot()                    #Än så länge kan man iaf kolla på bilden och se att det är olika spectrum "över" kärnan och skalet.